In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import *

In [3]:
class ContrapositiveLoss(losses.Loss):
    def __init__(self, margin=1):
        super().__init__()
        self.margin = margin
    
    def call(self, y_true, y_pred):
        margin_sq = np.max(margin - y_pred, 0)
        return np.mean(y_true * np.square(y_pred) + (1 - y_true) * np.square(margin_sq))